# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21097019


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:37,  3.48s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<01:01,  2.45s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:39,  1.74s/it]

Rendering models:  30%|███       | 9/30 [00:07<00:25,  1.23s/it]

...

Rendering models:  67%|██████▋   | 20/30 [00:07<00:03,  3.05it/s]

Rendering models:  77%|███████▋  | 23/30 [00:07<00:01,  4.09it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  5.35it/s]

Rendering models:  97%|█████████▋| 29/30 [00:08<00:00,  7.03it/s]

not-logged-in-3b0b929f4475fd276561    0.000762
Jnursssmith5263                       0.002372
martianmax                            0.000155
Jonas_Cross                           0.000161
not-logged-in-42aeec5653762437f6b7    0.000191
not-logged-in-c6917d9b19f41e6c2995    0.000620
awright5                              0.000500
SciKid10                              0.001459
Username12345678                      0.030300
not-logged-in-d7c7d6d93e6fb6e1814e    0.006373
girlsclub2018                         0.000293
not-logged-in-9a45911eb71184fe7c7f    0.001174
not-logged-in-f82a6d15a1dafa9bbfae    0.000341
pangeli5                              0.000292
TiaIsHere                             0.001275
peakscience7                          0.000194
Planetme                              0.000523
froggler                              0.000179
acapirala                             0.000226
jordanrushworth                       0.004362
cowslip                               0.004741
ts135992     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())